In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve, roc_auc_score
from scipy.ndimage.filters import uniform_filter1d

<ipython-input-1-cd05775307e9>:6: DeprecationWarning: Please use `uniform_filter1d` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import uniform_filter1d


In [2]:

from keras.models import Sequential, Model
from keras.layers import Conv1D, MaxPool1D, Dense, Dropout, Flatten, \
BatchNormalization, Input, concatenate, Activation
from keras.optimizers import Adam

In [3]:
import numpy as np


raw_data = np.loadtxt('exoPlanets.csv', skiprows=1, delimiter=',')

x = raw_data[:, 1:]
y = raw_data[:, 0, np.newaxis] - 1.
split_ratio = 0.8
split_index = int(len(x) * split_ratio)

x_train = x[:split_index]
y_train = y[:split_index]

x_test = x[split_index:]
y_test = y[split_index:]

In [4]:
x_train = ((x_train - np.mean(x_train, axis=1).reshape(-1,1)) /
           np.std(x_train, axis=1).reshape(-1,1))
x_test = ((x_test - np.mean(x_test, axis=1).reshape(-1,1)) /
          np.std(x_test, axis=1).reshape(-1,1))

In [5]:
x_train = np.stack([x_train, uniform_filter1d(x_train, axis=1, size=100)], axis=2)
x_test = np.stack([x_test, uniform_filter1d(x_test, axis=1, size=100)], axis=2)

In [6]:
model = Sequential()
model.add(Conv1D(filters=8, kernel_size=11, activation='relu', input_shape=x_train.shape[1:]))
model.add(MaxPool1D(strides=4))
model.add(BatchNormalization())
model.add(Conv1D(filters=16, kernel_size=11, activation='relu'))
model.add(MaxPool1D(strides=4))
model.add(BatchNormalization())
model.add(Conv1D(filters=32, kernel_size=11, activation='relu'))
model.add(MaxPool1D(strides=4))
model.add(BatchNormalization())
model.add(Conv1D(filters=64, kernel_size=11, activation='relu'))
model.add(MaxPool1D(strides=4))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [7]:
def batch_generator(x_train, y_train, batch_size=32):
    half_batch = batch_size // 2
    x_batch = np.empty((batch_size, x_train.shape[1], x_train.shape[2]), dtype='float32')
    y_batch = np.empty((batch_size, y_train.shape[1]), dtype='float32')

    yes_idx = np.where(y_train[:,0] == 1.)[0]
    non_idx = np.where(y_train[:,0] == 0.)[0]

    while True:
        np.random.shuffle(yes_idx)
        np.random.shuffle(non_idx)

        x_batch[:half_batch] = x_train[yes_idx[:half_batch]]
        x_batch[half_batch:] = x_train[non_idx[half_batch:batch_size]]
        y_batch[:half_batch] = y_train[yes_idx[:half_batch]]
        y_batch[half_batch:] = y_train[non_idx[half_batch:batch_size]]

        for i in range(batch_size):
            sz = np.random.randint(x_batch.shape[1])
            x_batch[i] = np.roll(x_batch[i], sz, axis = 0)

        yield x_batch, y_batch

In [8]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler

# Define a learning rate schedule
def lr_schedule(epoch):
    if epoch < 10:
        return 1e-4
    else:
        return 1e-5

# Create a learning rate scheduler
lr_scheduler = LearningRateScheduler(lr_schedule)

# Compile the model with the initial learning rate
model.compile(optimizer=Adam(1e-4), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model with the learning rate scheduler
history = model.fit_generator(batch_generator(x_train, y_train, 32),
                              validation_data=(x_test, y_test),
                              verbose=1, epochs=50, callbacks=[lr_scheduler],
                              steps_per_epoch=x_train.shape[0] // 32)


Epoch 1/50


<ipython-input-8-cbcb90e15b24>:18: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(batch_generator(x_train, y_train, 32),


127/127 [==============================] - 24s 125ms/step - loss: 0.7196 - accuracy: 0.5502 - val_loss: 0.7677 - val_accuracy: 0.0845 - lr: 1.0000e-04
Epoch 2/50
127/127 [==============================] - 11s 89ms/step - loss: 0.6698 - accuracy: 0.6085 - val_loss: 0.8109 - val_accuracy: 0.2417 - lr: 1.0000e-04
Epoch 3/50
127/127 [==============================] - 10s 80ms/step - loss: 0.6353 - accuracy: 0.6326 - val_loss: 0.7445 - val_accuracy: 0.4862 - lr: 1.0000e-04
Epoch 4/50
127/127 [==============================] - 11s 85ms/step - loss: 0.5976 - accuracy: 0.6848 - val_loss: 0.6158 - val_accuracy: 0.6503 - lr: 1.0000e-04
Epoch 5/50
127/127 [==============================] - 12s 91ms/step - loss: 0.5658 - accuracy: 0.7111 - val_loss: 0.5818 - val_accuracy: 0.6778 - lr: 1.0000e-04
Epoch 6/50
127/127 [==============================] - 11s 90ms/step - loss: 0.5242 - accuracy: 0.7416 - val_loss: 0.5423 - val_accuracy: 0.7181 - lr: 1.0000e-04
Epoch 7/50
127/127 [========================

In [9]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler

# Define a learning rate schedule
def lr_schedule(epoch):
    if epoch < 10:
        return 4e-5
    else:
        return 2e-5

# Create a learning rate scheduler
lr_scheduler = LearningRateScheduler(lr_schedule)

# Compile the model with the initial learning rate
model.compile(optimizer=Adam(4e-5), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model with the learning rate scheduler
history = model.fit_generator(batch_generator(x_train, y_train, 32),
                              validation_data=(x_test, y_test),
                              verbose=2, epochs=40, callbacks=[lr_scheduler],
                              steps_per_epoch=x_train.shape[0] // 32)


Epoch 1/40


<ipython-input-9-4bbf9b3adba9>:18: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(batch_generator(x_train, y_train, 32),


127/127 - 13s - loss: 0.2497 - accuracy: 0.9021 - val_loss: 0.2326 - val_accuracy: 0.8900 - lr: 4.0000e-05 - 13s/epoch - 102ms/step
Epoch 2/40
127/127 - 10s - loss: 0.2355 - accuracy: 0.9102 - val_loss: 0.2541 - val_accuracy: 0.8792 - lr: 4.0000e-05 - 10s/epoch - 80ms/step
Epoch 3/40
127/127 - 10s - loss: 0.2331 - accuracy: 0.9129 - val_loss: 0.2594 - val_accuracy: 0.8752 - lr: 4.0000e-05 - 10s/epoch - 81ms/step
Epoch 4/40
127/127 - 10s - loss: 0.2248 - accuracy: 0.9166 - val_loss: 0.2158 - val_accuracy: 0.8988 - lr: 4.0000e-05 - 10s/epoch - 77ms/step
Epoch 5/40
127/127 - 9s - loss: 0.2230 - accuracy: 0.9163 - val_loss: 0.2233 - val_accuracy: 0.8929 - lr: 4.0000e-05 - 9s/epoch - 73ms/step
Epoch 6/40
127/127 - 10s - loss: 0.2207 - accuracy: 0.9156 - val_loss: 0.2030 - val_accuracy: 0.9037 - lr: 4.0000e-05 - 10s/epoch - 80ms/step
Epoch 7/40
127/127 - 10s - loss: 0.2201 - accuracy: 0.9176 - val_loss: 0.2074 - val_accuracy: 0.9018 - lr: 4.0000e-05 - 10s/epoch - 80ms/step
Epoch 8/40
127/127

In [11]:
final_stats = model.evaluate(x_test, y_test, verbose=2)
print("Final Test Loss:", final_stats[0])
print("Final Test Accuracy:", final_stats[1])


32/32 - 1s - loss: 0.1211 - accuracy: 0.9499 - 647ms/epoch - 20ms/step
Final Test Loss: 0.12105606496334076
Final Test Accuracy: 0.9499017596244812
